In [ ]:
Tweet_tokeniser -> glove -> LSTM/GRU/BERT


In [6]:
import numpy as np 
import pandas as pd 
import os, math
 
#for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
 
#for text cleaning
import string, re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
 
#for data analysis and modeling
import tensorflow as tf
import tensorflow_hub as hub
!pip install tensorflow_text
import tensorflow_text 
from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.metrics import Recall
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Embedding, Dropout
from tensorflow.keras.layers import TextVectorization
tf.__version__
# 2.12.0
 
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

[nltk_data] Downloading package punkt to /home/benjamin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/benjamin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ImportError: /home/benjamin/anaconda3/lib/python3.11/site-packages/tensorflow_text/core/pybinds/tflite_registrar.so: undefined symbol: _ZN4absl12lts_2023080216raw_log_internal21internal_log_functionB5cxx11E

In [ ]:
data = pd.read_csv(
    '/home/benjamin/Documents/OpenClassroomsDatasets/sentiment/sentiment140/training.1600000.processed.noemoticon.csv',
    encoding = "ISO-8859-1",
    names=["target", "id", "date", "flag", "user", "text"]
)

data = data.drop(columns=["id", "date", "flag", "user"])

data.target = data.target.map(
    {
        0: 0.0,
        2: 0.0,
        4: 1.0,
    }
)

def sample_equal_classes(df, n_pos=100000, n_neg=100000):
    df_pos = df[df["target"] == 1.0].sample(n=n_pos)
    df_neg = df[df["target"] == 0.0].sample(n=n_neg)
    
    return pd.concat([df_pos, df_neg]).reset_index(drop=True)
sampled_df = sample_equal_classes(data)

xtrain, xtest, ytrain, ytest = train_test_split(sampled_df['text'].values, sampled_df['target'].values, shuffle=True, test_size=0.2)


In [ ]:
max_words = 20000